In [17]:
import pandas as pd
import folium
import requests
import os
import dask
import glob
from time import sleep
from dask import delayed
from dask.distributed import Client
import dask.dataframe as dd

In [18]:
df_station = pd.read_csv("./isd-history.csv")
df_station
open_station = df_station[df_station["END"] == 20220502]

In [19]:
dtypes = {
'STATION':         int,
'DATE':           object,
'SOURCE':         object,
'LATITUDE':      float,
'LONGITUDE':     float,
'ELEVATION':     float,
'NAME':           object,
'REPORT_TYPE':    object,
'CALL_SIGN':      object,
'QUALITY_CONTROL':object,
'WND':            object,
'CIG':            object,
'VIS':            object,
'TMP':            object,
'DEW':            object,
'SLP':            object,
'AA1':            object,
'AT1':            object,
'AT2':            object,
'AT3':            object,
'AT4':            object,
'AT5':            object,
'AU1':            object,
'AW1':            object,
'OC1':            object,
'REM':            object,
'EQD':            object
}

In [20]:
client = Client(n_workers=4)

/opt/homebrew/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51246 instead
  warnings.warn(


In [21]:
if not os.path.exists("./data"):
    os.mkdir("./data")

for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    if not os.path.exists(path):
        os.mkdir(path)

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if not os.path.exists(path + "/" + station + ".csv"):
            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

            if (requests.get(url).status_code == 200):
                #df = pd.read_csv(url)
                ddf = dd.read_csv(url, dtype='object')
                ddf.to_csv(path + "/" + station + ".csv", index=True, single_file = True)
                #df = pd.read_csv(url).to_csv(path + "/" + station + ".csv", index=True)
                #sleep(1)

2022-05-06 09:48:31,105 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-05-06 09:48:31,199 - distributed.nanny - ERROR - Worker process died unexpectedly


KeyboardInterrupt: 

2022-05-06 09:48:31,199 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-05-06 09:48:31,200 - distributed.nanny - ERROR - Worker process died unexpectedly
Exception in thread Nanny stop queue watch:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
2022-05-06 09:48:31,200 - distributed.nanny - ERROR - Worker process died unexpectedly
    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/lib/python3.9/site-packages/distributed/nanny.py", line 860, in watch_stop_q
    child_stop_q.close()
  File "/opt/homebrew/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 143, in close
    self._reader.close()
  File "/opt/homebre

In [22]:
for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if os.path.exists(path + "/" + station + ".csv"):
            #df = pd.read_csv(path + "/" + station + ".csv")
            #print(df.dtypes)
            df = dd.read_csv(path + "/" + station + ".csv", dtype=dtypes)
            
            
            #df = dd.read_csv(path + "/" + station + ".csv")
            #df.head()


In [23]:
dfs = []

for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    for filename in glob.glob(path + "/*.csv"):
        dfs.append(pd.read_csv(filename))

big_frame = pd.concat(dfs)
big_frame

,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,GF1,KA1,KA2,KA3,KA4,MA1,MW1,OC1,REM,EQD
0,0,72011354829,2022-01-01T00:15:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,"99,99,9,99,9,99,9,00488,1,99,9,99,9",NaN,NaN,NaN,NaN,"10064,1,99999,9",NaN,NaN,MET074METAR KVLL 010015Z AUTO 09006KT 10SM OVC...,NaN
1,1,72011354829,2022-01-01T00:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,"99,99,9,99,9,99,9,00488,1,99,9,99,9",NaN,NaN,NaN,NaN,"10068,1,99999,9",NaN,NaN,MET074METAR KVLL 010035Z AUTO 08004KT 10SM OVC...,NaN
2,2,72011354829,2022-01-01T00:55:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,"99,99,9,99,9,99,9,00488,1,99,9,99,9",NaN,NaN,NaN,NaN,"10064,1,99999,9",NaN,NaN,MET073METAR KVLL 010055Z AUTO 09004KT 7SM OVC0...,NaN
3,3,72011354829,2022-01-01T01:15:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,"99,99,9,99,9,99,9,00427,1,99,9,99,9",NaN,NaN,NaN,NaN,"10064,1,99999,9",NaN,NaN,MET073METAR KVLL 010115Z AUTO 08004KT 7SM OVC0...,NaN
4,4,72011354829,2022-01-01T01:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,"99,99,9,99,9,99,9,00366,1,99,9,99,9",NaN,NaN,NaN,NaN,"10058,1,99999,9",NaN,NaN,MET076METAR KVLL 010135Z AUTO 09006KT 5SM BR O...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688,72011354829,2022-05-02T06:15:00,6,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,KVLL,...,NaN,NaN,NaN,NaN,NaN,"10132,5,09868,5",NaN,"0072,5",MET08905/02/22 01:15:02 METAR KVLL 020615Z 240...,NaN
8689,8689,72011354829,2022-05-02T06:35:00,6,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,KVLL,...,NaN,NaN,NaN,NaN,NaN,"10132,5,09868,5",NaN,"0077,5",MET08905/02/22 01:35:02 METAR KVLL 020635Z 250...,NaN
8690,8690,72011354829,2022-05-02T06:55:00,6,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,KVLL,...,NaN,NaN,NaN,NaN,NaN,"10132,5,09868,5",NaN,NaN,MET09305/02/22 01:55:02 METAR KVLL 020655Z 250...,NaN
8691,8691,72011354829,2022-05-02T07:15:00,6,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,KVLL,...,NaN,NaN,NaN,NaN,NaN,"10132,5,09868,5",NaN,NaN,MET09305/02/22 02:15:02 METAR KVLL 020715Z 250...,NaN


# Clean dataframe

In [24]:
big_frame['TMP_control'] = big_frame['TMP'].str.split(pat=',', expand=True)[1]
big_frame['TMP_test'] = big_frame['TMP'].str.split(pat=',', expand=True)[0]
big_frame['TMP_sign'] = big_frame['TMP'].astype(str).str[0]
big_frame['TMP_final'] = (big_frame['TMP_test'].astype(str).str[1:4] + "." + big_frame['TMP_test'].astype(str).str[4:]).astype(float)
big_frame['DATE'] = pd.to_datetime(big_frame['DATE'], format="%Y-%m-%dT%H:%M:%S")
big_frame = big_frame[big_frame['TMP_control'] == "1"]

In [25]:
big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']
big_frame

/var/folders/mb/dct658p12cs6fnf1h3084gph0000gn/T/ipykernel_7310/46208713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
/var/folders/mb/dct658p12cs6fnf1h3084gph0000gn/T/ipykernel_7310/46208713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']


,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,KA4,MA1,MW1,OC1,REM,EQD,TMP_control,TMP_test,TMP_sign,TMP_final
0,0,72011354829,2022-01-01 00:15:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10064,1,99999,9",NaN,NaN,MET074METAR KVLL 010015Z AUTO 09006KT 10SM OVC...,NaN,1,+0048,1,4.8
1,1,72011354829,2022-01-01 00:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10068,1,99999,9",NaN,NaN,MET074METAR KVLL 010035Z AUTO 08004KT 10SM OVC...,NaN,1,+0047,1,4.7
2,2,72011354829,2022-01-01 00:55:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10064,1,99999,9",NaN,NaN,MET073METAR KVLL 010055Z AUTO 09004KT 7SM OVC0...,NaN,1,+0048,1,4.8
3,3,72011354829,2022-01-01 01:15:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10064,1,99999,9",NaN,NaN,MET073METAR KVLL 010115Z AUTO 08004KT 7SM OVC0...,NaN,1,+0048,1,4.8
4,4,72011354829,2022-01-01 01:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10058,1,99999,9",NaN,NaN,MET076METAR KVLL 010135Z AUTO 09006KT 5SM BR O...,NaN,1,+0047,1,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8607,8607,72011354829,2022-05-01 03:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10142,1,99999,9",NaN,"0077,1",MET085METAR KVLL 010335Z AUTO 09010G15KT 10SM ...,NaN,1,+0116,1,11.6
8608,8608,72011354829,2022-05-01 03:55:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10142,1,99999,9",NaN,"0082,1",MET093METAR KVLL 010355Z AUTO 09009G16KT 060V1...,NaN,1,+0118,1,11.8
8609,8609,72011354829,2022-05-01 04:15:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10139,1,99999,9",NaN,"0077,1",MET078METAR KVLL 010415Z AUTO 10009G15KT 10SM ...,NaN,1,+0118,1,11.8
8610,8610,72011354829,2022-05-01 04:35:00,4,42.54306,-83.17806,218.2,"OAKLAND TROY AIRPORT, MI US",FM-15,99999,...,NaN,"10135,1,99999,9",NaN,"0077,1",MET085METAR KVLL 010435Z AUTO 09008G15KT 10SM ...,NaN,1,+0117,1,11.7


In [26]:
cleaned_frame = big_frame[["NAME", "TMP_final", "LATITUDE", "LONGITUDE", "DATE"]]
cleaned_frame

,NAME,TMP_final,LATITUDE,LONGITUDE,DATE
0,"OAKLAND TROY AIRPORT, MI US",4.8,42.54306,-83.17806,2022-01-01 00:15:00
1,"OAKLAND TROY AIRPORT, MI US",4.7,42.54306,-83.17806,2022-01-01 00:35:00
2,"OAKLAND TROY AIRPORT, MI US",4.8,42.54306,-83.17806,2022-01-01 00:55:00
3,"OAKLAND TROY AIRPORT, MI US",4.8,42.54306,-83.17806,2022-01-01 01:15:00
4,"OAKLAND TROY AIRPORT, MI US",4.7,42.54306,-83.17806,2022-01-01 01:35:00
...,...,...,...,...,...
8607,"OAKLAND TROY AIRPORT, MI US",11.6,42.54306,-83.17806,2022-05-01 03:35:00
8608,"OAKLAND TROY AIRPORT, MI US",11.8,42.54306,-83.17806,2022-05-01 03:55:00
8609,"OAKLAND TROY AIRPORT, MI US",11.8,42.54306,-83.17806,2022-05-01 04:15:00
8610,"OAKLAND TROY AIRPORT, MI US",11.7,42.54306,-83.17806,2022-05-01 04:35:00


# Statistiques globales de température (moy, min, max)

In [27]:
temp = cleaned_frame.groupby(by="NAME").agg({'TMP_final': ['mean', 'min', 'max'], "LATITUDE": "first", "LONGITUDE": "first"})
temp

TMP_final             LATITUDE LONGITUDE
                                 mean  min   max     first     first
NAME                                                                
OAKLAND TROY AIRPORT, MI US  5.533735 -6.5  14.2  42.54306 -83.17806

# Affichage Station (MAP)

In [47]:
m = folium.Map(location=[temp["LATITUDE"].mean(), temp["LONGITUDE"].mean()], zoom_start=9)

for i in range(0,len(temp)):
   folium.Marker(
      location=[temp.iloc[i]['LATITUDE'], temp.iloc[i]['LONGITUDE']],
      popup=folium.Popup(temp.iloc[i].to_frame().to_html(), max_width=450,min_height=400)
   ).add_to(m)
   
m

In [28]:
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year

#    if not os.path.exists(path):
#        os.mkdir(path)

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban

#        if not os.path.exists(path + "/" + station + ".csv"):
#            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

#            if (requests.get(url).status_code == 200):
#                pd.read_csv(url).to_csv(path + "/" + station + ".csv")

#            spark.sparkContext.addFile('https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv')

#            if (SparkFiles.get(station + ".csv")):
#                df = spark.read.csv("file://"+SparkFiles.get(station + ".csv"), header=True)

#                print("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")
#                df.write.format("csv").save("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")

#for year in range(2022, 2023):

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban
        
#        df = spark.read.csv("hdfs://namenode:9000/noaa/" + str(year) + "/" + station + ".csv", header=True)
#df = spark.read.csv("hdfs://namenode:9000/noaa/2022/72026853882.csv", header=True)
#df.take(5)